# Retrevial Agumented Generation (RAG)

Keeping large amounts of embedded data in a database and retreving the relevant informaiton for answering a question

### Indexing : Document loading with PyPDF Loader

In [54]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import CharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings

import numpy as np

import copy
import dotenv

dotenv.load_dotenv()


True

In [2]:
loader_pdf = PyPDFLoader("Documents_for_indexing/introduction_to_Data_and_Data_Science.pdf")

In [3]:
page_pdf = loader_pdf.load()

In [4]:
page_pdf

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-09T10:16:34+02:00', 'author': 'Hristina  Hristova', 'moddate': '2023-11-09T10:16:34+02:00', 'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious t

In [5]:
len(page_pdf)

6

In [6]:
pages_pdf_cut = copy.deepcopy(page_pdf)

In [9]:
# Remove all newline characters from the page content to reduce token count
for i in pages_pdf_cut:
    i.page_content = ' '.join(i.page_content.split())



In [11]:
page_pdf[0].page_content, pages_pdf_cut[0].page_content
# 375 tokens vs. 300 tokens (check at platform.openai.com/tokenizer)

('Analysis vs Analytics \nAlright! So… \nLet’s discuss the not-so-obvious differences \nbetween the terms analysis and analytics. \nDue to the similarity of the words, some people \nbelieve they share the same meaning, and thus \nuse them interchangeably. Technically, this \nisn’t correct. There is, in fact, a distinct \ndifference between the two. And the reason \nfor one often being used instead of the other \nis the lack of a transparent understanding \nof both. \nSo, let’s clear this up, shall we? \nFirst, we will start with analysis. \nConsider the following… \nYou have a huge dataset containing data of \nvarious types. Instead of tackling the entire \ndataset and running the risk of becoming overwhelmed, \nyou separate it into easier to digest chunks \nand study them individually and examine how \nthey relate to other parts. And that’s analysis \nin a nutshell. \nOne important thing to remember, however, \nis that you perform analyses on things that \nhave already happened in the

### Indexing : Document loading with DOX2TEXT Loader

In [13]:
loader_docx = Docx2txtLoader("Documents_for_indexing/introduction_to_Data_and_Data_Science.docx")

In [14]:
page_docx = loader_docx.load()

In [16]:
page_docx

[Document(metadata={'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.docx'}, page_content="Analysis vs Analytics\n\nAlright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to rem

In [18]:
for i in range(len(page_docx)):
    page_docx[i].page_content = ' '.join(page_docx[i].page_content.split())

In [22]:
len(page_docx[0].page_content)

8259

In [28]:
character_splitter = CharacterTextSplitter(
    separator=' ', 
    chunk_size=500, 
    chunk_overlap=50)

In [29]:
pages_character_split = character_splitter.split_documents(page_docx)

In [30]:
pages_character_split

[Document(metadata={'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.docx'}, page_content='Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will'),
 Document(metadata={'source': 'Documents_for_indexing/introduction_to_Data_and_Data_Science.docx'}, page_content='So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to dige

In [31]:
len(pages_character_split)

19

### Indexing : Document splitting with Markdown Header Test Splitter

In [34]:
loader_docx_2 = Docx2txtLoader("Documents_for_indexing/introduction_to_Data_and_Data_Science_2.docx")
page_docx_2 = loader_docx_2.load()

In [35]:
from httpx import head


md_header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Course Title"),
        ("##", "Lecture Title")
    ]
)

In [36]:
pages_md_split = md_header_splitter.split_text(page_docx_2[0].page_content)

In [37]:
pages_md_split

[Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content="Alright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to remember, however

In [38]:
for i in range(len(pages_md_split)):
    pages_md_split[i].page_content = ' '.join(pages_md_split[i].page_content.split())


In [41]:
pages_md_split

[Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content="Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individually and examine how they relate to other parts. And that’s analysis in a nutshell. One important thing to remember, however, is that you perform

### Indexing : Text Embedding with OpenAI

In [42]:
pages_md_and_char_split = character_splitter.split_documents(pages_md_split)

In [43]:
pages_md_and_char_split

[Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis.'),
 Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to dige

In [48]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [50]:
pages_md_and_char_split[3], pages_md_and_char_split[5], pages_md_and_char_split[18]

(Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='ones. Analytics is essentially the application of logical and computational reasoning to the component parts obtained in an analysis. And in doing this you are looking for patterns and exploring what you could do with them in the future. Here, analytics branches off into two areas: qualitative analytics – this is using your intuition and experience in conjunction with the analysis to plan your next business move. And quantitative analytics – this is applying formulas and algorithms to numbers'),
 Document(metadata={'Course Title': 'Introduction to data science', 'Lecture Title': 'Analysis vs Analytics'}, page_content='styles of clothing to start selling. This would be qualitative analytics. But you might not know when to introduce the new collection. In that case, relying on past sales data and user experience data, you could predict in which month it would be be

In [53]:
vector_1 = embeddings.embed_query(pages_md_and_char_split[3].page_content)
vector_2 = embeddings.embed_query(pages_md_and_char_split[5].page_content)  
vector_3 = embeddings.embed_query(pages_md_and_char_split[18].page_content)

In [55]:
np.dot(vector_1, vector_2)

np.float64(0.8605078118583023)

In [56]:
np.dot(vector_1, vector_3)

np.float64(0.7641135414930623)

In [57]:
np.dot(vector_2, vector_3)

np.float64(0.7654245657809027)

In [58]:
np.linalg.norm(vector_1) # check the magnitude (length) of the vector

np.float64(1.0000000362284815)

In [59]:
np.linalg.norm(vector_2) 

np.float64(0.9999999874541907)